# Boosting and Advanced Sklearn

<b>Goals</b>

- Follow up our lesson on ensemble methods with boosting, what it is and how it works.
- Use the super Gradient Boosting Classifier/Regressor.
- How to use high-powered tools in sklearn to optimize your models and minimize your work load and time

## Boosting

- Boosting is an ensemble method where a model is comprised of a sequence of models, as opposed to a set of parallel models as with Random Forest.
- Unlike bagging, boosting uses random subsets of training data <b>WITHOUT</b> replacement.
- It is an iterative process. Begins by training simple model on the whole data, pinpoints the inaccuracies, and trains a new model to target those inaccuracies (misclassification rate, residuals.) The new models try to predict what the previous ones were unable to correctly predict. Repeat until reaching a stopping point parameter. The whole set of models is what's used to make predictions.
- Boosting process:
    - Randomly select a batch of data from training dataset without replacement to train "weak learner."
    - Randomly select a second batch of data from training dataset without replacement AND add around half of the samples that were misclassified from the previous model.
    - Go back to the original training dataset and retrieve the data points in which the two models had differing classifications.
    - Make predictions by combining the system of weak learners and takin the vote (classification) or avearge (regression.)
    
- Can be used both for regression and classification

### AdaBoost Classifier

- The AdaBoost (Adaptive Boosting) algorithm fits sequential weak classifiers, which are classifiers that are slightly better than random chance. These classifiers are usually tree-based models with a lower depth level. Adaboost actually uses the whole training dataset instead of sample. The data is weighted in each iteration of modeling to help it learn from the mistakes of the previous models.

- The weak learners in the AdaBoost algorithm are Decision Trees with one depth-level aka "Decision Stumps." They literally only use one decision.

- Each data point in the training data is assigned a weight. In the first model, every point has the same weight value which equal 1/number of values. 

- The first Decision Stump is fit on the whole data using weighted samples. Only works with binary clasification problems. The model outputs either a 1 or - 1, irregardless of the class labels in the target variable. 

- Error determined by the misclassification rate, which is 1 - accuracy score. Accuracy score of 0.71 means error rate of 0.29.

- However error significantly changes when differents are introduced. 

- With weights, error = sum(w(i) * terror(i)) / sum(w). If terror is 1, then equals wrong prediction, 0 if correct.

AdaBoost visually explained:

![a](https://www.analyticsvidhya.com/wp-content/uploads/2015/11/bigd.png)

Source: [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2015/11/quick-introduction-boosting-algorithms-machine-learning/)

Box 1: Each data point has equal weighting is fit on a decision stump which is a vertical line.

<br>

Box 2: The three plus signs that were incorrectly classified in Box 1 have been enlarged (weighted) and the model has been retrained.

<br>

Box 3: Three minus signs have been given bigger weight values and the new model (horizontal line) has been fit to account for that.

<br>

Box 4: Combines the three Decision stump models, which vastly outperforms any of the three stumps.

Let's manually calculate weights

In [ ]:
#List of weights
w = [0.2, 0.2, 0.2, 0.2, 0.2]
#List of actual values
y = [1,  1, -1, 1, -1]
#List of predictions
p = [-1, 1, 1, 1, -1]
#List or terrors
t = [1, 0, 1, 0 , 0]

In [ ]:
#Regular error rate calculation
(1 + 0 + 1 + 0 + 0)/(1 + 1 + 1 + 1 + 1.)

In [ ]:
#Error calculation with weights (same product as above)
e = (0.2 * 1 + 0.2 * 0 + 0.2 * 1 + 0.2 * 0 + 0.2 * 0)/ (0.2 + 0.2 + 0.2 + 0.2 + 0.2)
e

In the next part we pass in the error rate through this function: 0.5 * log((1-e)/e)

This gives a coefficient: a

In [ ]:
#Import numpy
import numpy as np

a = 0.5 * np.log((1 - e)/ e)
a

We use this value to update our new weights.

Formula is old weight value times the exponent of the negative value of a times prediction times actual value

In [ ]:
#First value

w1 = 0.2 * np.exp(-a * 1 * -1)
w1

w2 = 0.2 * np.exp(-a * 1 * 1)
w2

w3 = 0.2 * np.exp(-a * 1 * -1)
w3

w4 = 0.2 * np.exp(-a * 1 * 1)
w4

w5 = 0.2 * np.exp(-a * 1 * 1)
w5

print (w1, w2, w3, w4, w5)

Weights go up for wrong predictions and go down for correct ones.

We're not finished yet.

Then we normalize the weight by diving each weight by the sum of weights

In [ ]:
weight_sum = w1 + w2 + w3 + w4 + w5

#New weights
w1 = w1/(weight_sum)
w2 = w2/(weight_sum)
w3 = w3/(weight_sum)
w4 = w4/(weight_sum)
w5 = w5/(weight_sum)

print (w1, w2, w3, w4, w5)

These are our new weights which we'll use in the next round of modeling

In the follow - up model, a second Decision Stump model is trained using our new weights. The weights are used to determine the split in the decision tree. This process continues until we reach the n_estimators parameter we set.

Increasing the weights for the mis-classified data points forces the models to train more heavily on the data it incorrectly classified.

### Predictions

- AdaBoost makes predictions by calculating the weighted average of the sequence of Decision Stumps. 

- When you pass in a new data point, the model predicts 1 or -1.

- The weights of each model by each one's stage value. The prediction is derived from the sum of the of the weighted predictions. If sum > 0 then return the first class else return second class.

In [ ]:
#Five model predictions
preds = np.array([-1, -1, 1, -1, 1])
preds.sum()

Without weighting the prediction would be -1.

In [ ]:
weights = np.array([.2, .4, .8, .3, .9])

sum(weights * preds)

Prediction with weighted models equals 1.

<b>Warnings</b>

- Requires rich data noisy data by design can negatively influence model.
- Same goes with outliers, the model will chase outliers.

### Coding AdaBoost

1. Visualize the decision boundaries of AdaBoost

2. Use AdaBoost on the spotify dataset

In [ ]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

Generate and visualize fake data

In [ ]:

#Generate fake data that is 400 x 2.
data = make_classification(n_samples=400, n_features=2, n_informative=2, n_redundant=0, 
                    class_sep=.54, random_state = 8)

df = pd.DataFrame(data[0], columns=["feature1", "feature2"])
#Add target variable to df 
df["target"] = data[1]

#Call scatter plot of feature1 vs feature2 with color-encoded target variable
plt.style.use("fivethirtyeight")
plt.figure(figsize=(11, 8))
#Color encode target variable
colors = df.target.map({0:"b", 1:"r"})
plt.scatter(df.feature1, df.feature2, c = colors, s = 100, alpha=.5);

In [ ]:
#Assign X and y


#Fit a Decision Tree model with max_depth = 1 on the data.



In [ ]:
#Decision boundary function
def plot_decision_boundary(model, X, y):
    X_max = X.max(axis=0)
    X_min = X.min(axis=0)
    xticks = np.linspace(X_min[0], X_max[0], 100)
    yticks = np.linspace(X_min[1], X_max[1], 100)
    xx, yy = np.meshgrid(xticks, yticks)
    ZZ = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = ZZ >= 0.5
    Z = Z.reshape(xx.shape)
    plt.rcParams["figure.figsize"] = (10,7)
    fig, ax = plt.subplots()
    ax = plt.gca()
    ax.contourf(xx, yy, Z, cmap=plt.cm.bwr, alpha=0.2)
    ax.scatter(X[:,0], X[:,1], c=y, alpha=0.4, s = 50)

In [ ]:
#Feed dt model, features and colors


In [ ]:
#Train AdaBoost model on the same data and visualize it

#Intialize AdaBoost model with 20 estimators


#Fit model


#Visualize model boundaries


Let's use AdaBoost on Spotify data

In [ ]:
#Import spotify data

spotify = pd.read_csv("../data/spotify_data.csv", index_col=[0])
spotify.head()

Compare and contrast Decision Trees and AdaBoost

In [ ]:
#Intialize Decision Stump


#Intialize AdaBoost with 300 estimators


In [ ]:
#Assign X and y

X = 

y = 

In [ ]:
#Null accuracy



In [ ]:
#Train test split with test size = 0.33 and random_state = 12

X_train, X_test, y_train, y_test = 

In [ ]:
#Fit Decision Stump on training data and score it on training and testing sets



In [ ]:
#Fit Ada boost model on training data and score it on testing



Cross validate both models

In [ ]:
#Cross validate DT

In [ ]:
#Cross validate ADA

Cross validation shows AdaBoost doesn't really improve our model.

Perhaps we chose the wrong estimator value.

Let's make a validation curve to determine the best value for the estimator.

<br>

This will take a while.

In [ ]:
#We're going to time our code

#Import time tool
from time import time

In [ ]:
#Intialize time variable
t = time()

#Create list of estimator values
estimators = 

#Intialize cross validation scores list
cv_scores = []

#Iterate over estimators values, fit models, and then append scores to cv_scores
for est in estimators:
    

    
#Print difference in time



In [ ]:
    
#Plot estimators versus scores



In [ ]:
#Derive best estimator value


What if I told you there was a more efficient way to find the best parameter for a given model? The best depth or K-neighbors, etc?

## Advanced Sklearn tools

Overview:

- Grid search
- Pipelinings
- Imputation
- Feature unions
- Feature selections

In [ ]:
#More imports
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline

### Gridsearch

Algorithm that tests every combination of model parameters to find the best one.

Let's use GridSearch to find the best K value for a KNN model and Spotify data

In [ ]:
#Intialize parameter grid

#Range of neighbors to test


#Dictionary of parameter values 


In [ ]:
#Intialize Grid


#Fit grid on data



In [ ]:
#The scores

In [ ]:
#Whats the best cross validated accuracy score



In [ ]:
#Find the best parameters



This simple technique gives us the best K value.

We can use the best model from grid_knn to make predictions.

In [ ]:
#Input 
x = [[0.2, .15, 0.68, 0.05, 0.328]]

#Make prediction 


In [ ]:
#Get probability


Quick exercise:
<br>
Use grid search to determine the best depth value in a decision tree. Use depths from 2 - 20.

In [ ]:


#Range of neighbors to test


#Dictionary of parameter values 


The CV in GridSearchCV stands for cross validation which means we have to set a cv and scoring value.

In [ ]:
#Intialize Grid


#Fit grid on data



In [ ]:
#Best score for DT model


In [ ]:
#Best parameter for DT model


In [ ]:
#Make prediction



So far our grids have been one-dimensional, now let's try using multiple dimensions

In [ ]:
#Param grid with test different split criteria as well.
param_grid_dt = 

param_grid_dt

It's going to cross-validate every combination between the criterion parameters and depth parameters.

In [ ]:
#Intialize Grid
grid_dt = 
#Fit grid on data


In [ ]:
#Best parameter



In [ ]:
#Best score



How many models did this grid search function conduct?

Let's add in some parameters

In [ ]:

param_grid_dt["min_samples_split"] = 
param_grid_dt["max_features"] = 

In [ ]:
#Intialize Grid
grid_dt = 

#Time the code 



#Fit grid on data


#Print time difference



In [ ]:
#Best parameter



In [ ]:
#Best score



In [ ]:
#Make prediction



Obviously grid search takes a long time and in some case can cause memory errors. This is where RandomizedSearchCV comes in.

In [ ]:
#Import 


Functions just like GridSearchCV, except we have to choose a value n_iter which is the random number of combinations we testing and set param_distributions instead of param_grid.

In [ ]:
#Intialize RandomizedSearchCV grid with n_iter = 20
grid_dt = 

#Time the code 

t = time()

#Fit grid on data


#Print time difference

print (time() - t)

Reduced run time by 96%!

But now let's see if we sacrificed performance. Remember our previous best score was 0.667.

In [ ]:
#Check best score



By going from about 24 seconds to 0.87 we sacrificed 0.009 percentage points in accuracy.

Good trade-off?

### Pipelines

Let's go back to using the KNN model.


We know that we need to scale our data for the KNN algorithm right?

In [ ]:
#Scale data and fit it a Grid search function it.

#Intialize scalar


#Fit and transform scaler on the data
Xs = 

In [ ]:
#Intialize Grid

grid_knn_s = 

#Fit grid on scaled data



In [ ]:
#Best score


In [ ]:
#Best K value


In [ ]:
#Make prediction

#First transform predict using scaler



#Pass in xs to grid model



Time to make a pipeline.

In [ ]:
#Pass scaler and knn classifier objects into make_pipeline function


In [ ]:
#Create new param_grid
neighbors_range = range(2, 21)
param_grid_knn = {}
param_grid_knn["kneighborsclassifier__n_neighbors"] = neighbors_range
param_grid_knn

In [ ]:

#Pass in pipe into GridSearchCV function, 
grid_knn_pipe = 

#Fit on original versions of data


#Best scores and params
print 

You can also pass in the pipe object into a cross_val_score function

In [ ]:
# Use the cross-validation process using Pipeline
pipe = 


Class exercise: Use grid search to model the 2016 Democratic primary data.

In [ ]:
#Load in data files
primary = pd.read_csv("../data/primary_data//primary_results.csv")
county = pd.read_csv("../data/primary_data/county_facts.csv")
county_dict = pd.read_csv("../data/primary_data/county_facts_dictionary.csv")

subset_col_index = [0,3,5,9,10,12,18,20,23,25,33,34,53]

county = county.iloc[:,subset_col_index].copy()

subset_cols = ["fips","population", "pop_change", "senior_pop_per", "female_pop_per", "black_pop_per",
               "white_pop_per", "foreign_pop_per", "college_degree_pop_", "commute_time", "median_income",
               "poverty_rate", "pop_density"]

col_dict = dict(zip(county.columns, subset_cols))
#Use dictionary to rename the columns
county.rename(columns=col_dict, inplace=True)
primary.dropna(inplace=True)
bern = primary[primary.candidate== "Bernie Sanders"]
hill = primary[primary.candidate== "Hillary Clinton"]
bern = bern[["fips", "candidate", "votes"]]
dem = pd.merge(hill, bern, on="fips")
dem.rename(columns={"votes_x":"clinton_votes", "votes_y":"sanders_votes"}, inplace=True)
dem["winner"] = dem.clinton_votes - dem.sanders_votes
def vote_winner(x):
    if x >0:
        return "H"
    elif x == 0:
        return "TIE"
    else:
        return "B"
    
dem["winner"] = dem.winner.apply(vote_winner)

dem = dem[dem.winner!= "TIE"]
dem = dem[["fips", "winner"]]
df = pd.merge(county, dem, on="fips")
df.set_index("fips", inplace=True)
df.head()

In [ ]:
#Answers go here

Now let's make a pipeline using regression

In [ ]:
#Imports
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

In [ ]:
#Load in boston dataset
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(boston["data"])
df.columns = boston["feature_names"]
df["MEDV"] = boston["target"]
df.head()

In [ ]:
#Assign X and y

X = df.drop("MEDV", axis =1)
y = df.MEDV

In [ ]:
#Use a Pipeline Class instead of function to establish pipeline
pipe_poly = 

In [ ]:
#Select a few features from X
XX = 

In [ ]:
#Intialize range values for poly
poly_range = 

#Intialize grid dictionary
param_grid_poly = {}

#Input grid values
param_grid_poly["polynomialfeatures__degree"] = 

#Establish the grid
poly_grid =

In [ ]:
# Fit on data



Randomized Grid Search with Ridge regression

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
pipe_poly = Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                            ('ridgeregression', Ridge())]) 

param_grid_ridge = {'polynomialfeatures__degree': [1, 2, 3, 4, 5],
              'ridgeregression__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

grid_ridge = RandomizedSearchCV(pipe_poly, param_distributions=param_grid_ridge, 
                                n_iter = 5 , cv = 5, scoring='neg_mean_squared_error')
grid_ridge.fit(XX, y)

In [ ]:
print (grid_ridge.best_params_, grid_ridge.best_score_)

## Resources

<b> Boosting: </b>

- https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/

- https://www.analyticsvidhya.com/blog/2015/11/quick-introduction-boosting-algorithms-machine-learning/

- http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/

- https://www.youtube.com/watch?v=Rm6s6gmLTdg&list=PLaslQpv_LmSKxSCBPdKWEI7lLHrTCeewl

- https://www.coursera.org/learn/practical-machine-learning/lecture/9mGzA/boosting

<b> Grid Search and Pipelines </b>

- https://chrisalbon.com/machine-learning/cross_validation_parameter_tuning_grid_search.html
- https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/
- https://www.youtube.com/watch?v=Gol_qOgRqfA
- https://chrisalbon.com/machine-learning/pipelines_with_parameter_optimization.html
- https://chrisalbon.com/machine-learning/hyperparameter_tuning_using_random_search.html
- https://machinelearningmastery.com/automate-machine-learning-workflows-pipelines-python-scikit-learn/
- https://www.civisanalytics.com/blog/workflows-in-python-using-pipeline-and-gridsearchcv-for-more-compact-and-comprehensive-code/

# In class work.

Choose one of the following dataset to work on modeling for the rest of class using the new models and tools we've learned in the past couple weeks.

<br>
Spotify, Dem Primary, KC Housing, Movie Metadata, HR Employee, Breast Cancer, Default, Mushrooms, Red win quality, Zillow starter, or Pokemon.